In [1]:
import os
%pwd

'/Users/vaidehipawar/Desktop/mlops/EndToEnd_DSproject/research'

In [2]:
os.chdir('../')
%pwd



'/Users/vaidehipawar/Desktop/mlops/EndToEnd_DSproject'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [14]:

from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)  ## Contains paths and URLs (e.g., from config.yaml)
        self.params = read_yaml(params_filepath)  ## Training hyperparameters (e.g., params.yaml)
        self.schema = read_yaml(schema_filepath)  ## Data schema for validation (e.g., schema.yaml)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig: 
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        create_directories([self.config.artifacts_root])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )

        return model_trainer_config

In [11]:
import pandas as pd
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib


In [19]:
class ModelTrainer:
    def __init__(self, config:  ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state =42)
        lr.fit(train_x, train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir, self.config.model_name))



In [20]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e


[2025-06-04 15:27:58,297: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-04 15:27:58,299: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-04 15:27:58,301: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-04 15:27:58,303: INFO: common: created directory at: artifacts]
[2025-06-04 15:27:58,303: INFO: common: created directory at: artifacts/model_trainer]
